In [1]:
import pandas as pd
import numpy as np
import os
import dill as pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from scipy.stats import loguniform
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def load_sentence_embeddings(model='DistilBERT', features_path='features/', filename='sentence_embeddings'):
    if model == 'DistilBERT':
        tst_df = pd.concat([pickle.load(open(os.path.join(features_path, file), 'rb')) for file in os.listdir(features_path) if filename in file])
        return tst_df

In [3]:
tst_df = load_sentence_embeddings(features_path="features_250/")

In [4]:
tst_df.shape

(22332, 2)

In [5]:
tst_df.head()

,sentence_embeddings,label
0,"[-0.185443714261055, -0.11448108404874802, -0....",0
1,"[-0.3724660873413086, 0.04101637750864029, -0....",0
2,"[-0.41084980964660645, -0.1713167279958725, -0...",0
3,"[-0.14235153794288635, 0.19862940907478333, -0...",0
4,"[-0.47683459520339966, -0.040994927287101746, ...",0


In [6]:
tst_df.label.value_counts()

0    14888
1     7444
Name: label, dtype: int64

In [7]:
features = np.array(tst_df.sentence_embeddings.tolist())

In [8]:
labels = tst_df['label']

In [9]:
model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [10]:
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)

In [11]:
search = RandomizedSearchCV(model, space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)

In [ ]:
# execute search
result = search.fit(features, labels)

In [ ]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
# execute search
result = search.fit(features, labels)

In [ ]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
model1 = LogisticRegression()
cv1 = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [ ]:
space1 = dict()
space1['solver'] = ['liblinear']
space1['penalty'] = ['none', 'l2']
space1['class_weight'] = ["balanced"]
space1['C'] = loguniform(1e-5, 100)

In [ ]:
search1 = RandomizedSearchCV(model1, space1, n_iter=100, scoring='f1', n_jobs=-1, cv=cv1, random_state=1)

In [ ]:
# execute search
result1 = search1.fit(features, labels)

In [ ]:
# summarize result
print('Best Score: %s' % result1.best_score_)
print('Best Hyperparameters: %s' % result1.best_params_)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
best_lr_clf = LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')

In [ ]:
best_lr_clf.fit(train_features, train_labels)

In [ ]:
predictions = best_lr_clf.predict(test_features)
probs = best_lr_clf.predict_proba(test_features)[:, 1]

In [ ]:
result_table = [["Classifier", "F1", "Accuracy", "AUC"]]

In [ ]:
result_table.append([
    "LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')",
    round(f1_score(test_labels, predictions), 3),
    round(accuracy_score(test_labels, predictions), 3),
    round(roc_auc_score(test_labels, probs), 3),
])

In [ ]:
result_table

In [ ]:
from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

# KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
knn_model = KNeighborsClassifier(n_jobs=-1)

In [19]:
knn_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [12]:
knn_space = dict()
knn_space['weights'] = ['distance', 'uniform']
knn_space['n_neighbors'] = list(range(5, 50, 5))
knn_space['algorithm'] = ['auto', 'ball_tree', 'kd_tree']
knn_space['p'] = [1, 2]

In [13]:
knn_search = RandomizedSearchCV(knn_model, knn_space, n_iter=100, scoring='f1', n_jobs=-1, cv=knn_cv, random_state=1)

In [14]:
# execute search
knn_result = knn_search.fit(features, labels)

In [16]:
# summarize result
print('Best Score: %s' % knn_result.best_score_)
print('Best Hyperparameters: %s' % knn_result.best_params_)

Best Score: 0.9108119417459216
Best Hyperparameters: {'weights': 'distance', 'p': 1, 'n_neighbors': 20, 'algorithm': 'ball_tree'}


In [20]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [21]:
best_knn_clf = KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)

In [22]:
best_knn_clf.fit(train_features, train_labels)

KNeighborsClassifier(algorithm='ball_tree', n_jobs=-1, n_neighbors=20, p=1,
                     weights='distance')

In [23]:
predictions = best_knn_clf.predict(test_features)
probs = best_knn_clf.predict_proba(test_features)[:, 1]

In [34]:
result_table = [["F1", "Accuracy", "AUC"]]

In [24]:
result_table.append([
    "KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)",
    round(f1_score(test_labels, predictions), 3),
    round(accuracy_score(test_labels, predictions), 3),
    round(roc_auc_score(test_labels, probs), 3),
])

In [25]:
result_table

[['Classifier', 'F1', 'Accuracy', 'AUC'],
 ["LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')",
  0.974,
  0.982,
  0.998],
 ["KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)",
  0.903,
  0.935,
  0.979]]

In [26]:
from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

+-------------------------------------------------------------------------------------------------+-------+------------+-------+
| Classifier                                                                                      |    F1 |   Accuracy |   AUC |
+=================================================================================================+=======+============+=======+
| LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')            | 0.974 |      0.982 | 0.998 |
+-------------------------------------------------------------------------------------------------+-------+------------+-------+
| KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1) | 0.903 |      0.935 | 0.979 |
+-------------------------------------------------------------------------------------------------+-------+------------+-------+


# DT

In [27]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [24]:
dt_space = dict()
dt_space['random_state'] = [1]
dt_space['criterion'] = ['gini', 'entropy']
dt_space['max_depth'] = [50, 100, 500, 1000]
dt_space['splitter'] = ['best', 'random']
dt_space['class_weight'] = [None, 'balanced']
dt_space['max_features'] = [None, 'auto', 'sqrt', 'log2']

In [25]:
dt_search = RandomizedSearchCV(dt_model, dt_space, n_iter=100, scoring='f1', n_jobs=-1, cv=dt_cv, random_state=1)

In [26]:
# execute search
dt_result = dt_search.fit(features, labels)

In [27]:
# summarize result
print('Best Score: %s' % dt_result.best_score_)
print('Best Hyperparameters: %s' % dt_result.best_params_)

Best Score: 0.7903791474939794
Best Hyperparameters: {'splitter': 'best', 'random_state': 1, 'max_features': None, 'max_depth': 100, 'criterion': 'entropy', 'class_weight': 'balanced'}


In [28]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [29]:
best_dt_clf = DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=1,
                                     max_features=None, max_depth=100, class_weight='balanced')

In [30]:
best_dt_clf.fit(train_features, train_labels)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=100, random_state=1)

In [31]:
predictions = best_dt_clf.predict(test_features)
probs = best_dt_clf.predict_proba(test_features)[:, 1]

In [42]:
result_table = [["F1", "Accuracy", "AUC"]]

In [32]:
result_table.append([
    "DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=1, max_features=None, max_depth=100, class_weight='balanced')",
    round(f1_score(test_labels, predictions), 3),
    round(accuracy_score(test_labels, predictions), 3),
    round(roc_auc_score(test_labels, probs), 3),
])

In [33]:
result_table

[['Classifier', 'F1', 'Accuracy', 'AUC'],
 ["LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')",
  0.974,
  0.982,
  0.998],
 ["KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)",
  0.903,
  0.935,
  0.979],
 ["DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=1, max_features=None, max_depth=100, class_weight='balanced')",
  0.787,
  0.858,
  0.839]]

In [34]:
from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------------+-------+
| Classifier                                                                                                                              |    F1 |   Accuracy |   AUC |
+=========================================================================================================================================+=======+============+=======+
| LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')                                                    | 0.974 |      0.982 | 0.998 |
+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------------+-------+
| KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)                                         | 0.903 |      0.

In [45]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.DataFrame(result_table[1:], columns=['Classifier', 'F1', 'Accuracy', 'AUC'])

,Classifier,F1,Accuracy,AUC
0,"LogisticRegression(C=4.9, class_weight='balanced', penalty='l2', solver='liblinear')",0.974,0.982,0.998
1,"KNeighborsClassifier(p=1, weights='distance', n_neighbors=20, algorithm='ball_tree', n_jobs=-1)",0.903,0.935,0.979
2,"DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=1, max_features=None, max_depth=100, class_weight='balanced')",0.787,0.858,0.839


# more iter on dt

In [46]:
from sklearn.tree import DecisionTreeClassifier

dt_more_model = DecisionTreeClassifier()
dt_more_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [47]:
dt_more_space = dict()
dt_more_space['random_state'] = [1]
dt_more_space['criterion'] = ['gini', 'entropy']
dt_more_space['max_depth'] = list(range(1, 101))
dt_more_space['splitter'] = ['best', 'random']
dt_more_space['class_weight'] = [None, 'balanced']
dt_more_space['max_features'] = [None, 'auto', 'sqrt', 'log2']

In [48]:
dt_more_search = RandomizedSearchCV(dt_more_model, dt_more_space, 
                                    n_iter=500, scoring='f1', n_jobs=-1, cv=dt_more_cv, random_state=1)

In [49]:
# execute search
dt_more_result = dt_more_search.fit(features, labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 15000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 889, in fit
    super().fit(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py", l

In [50]:
# summarize result
print('Best Score: %s' % dt_more_result.best_score_)
print('Best Hyperparameters: %s' % dt_more_result.best_params_)

Best Score: 0.796373652263697
Best Hyperparameters: {'splitter': 'best', 'random_state': 1, 'max_features': None, 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced'}


In [51]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [52]:
best_dt_more_clf = DecisionTreeClassifier(criterion='gini', splitter='best', random_state=1,
                                     max_features=None, max_depth=10, class_weight='balanced')

In [53]:
best_dt_more_clf.fit(train_features, train_labels)

DecisionTreeClassifier(class_weight='balanced', max_depth=10, random_state=1)

In [54]:
predictions = best_dt_more_clf.predict(test_features)
probs = best_dt_more_clf.predict_proba(test_features)[:, 1]

In [55]:
result_table = [["F1", "Accuracy", "AUC"]]

In [56]:
result_table.append([
    round(f1_score(test_labels, predictions), 3),
    round(accuracy_score(test_labels, predictions), 3),
    round(roc_auc_score(test_labels, probs), 3),
])

In [57]:
result_table

[['F1', 'Accuracy', 'AUC'], [0.786, 0.852, 0.823]]

In [58]:
from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

+-------+------------+-------+
|    F1 |   Accuracy |   AUC |
+=======+============+=======+
| 0.786 |      0.852 | 0.823 |
+-------+------------+-------+


# RF

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_jobs=25)

In [10]:
rf_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [19]:
rf_space = dict()
rf_space['random_state'] = [1]
rf_space['criterion'] = ['gini', 'entropy']
rf_space['max_depth'] = list(range(1, 101, 10))
rf_space['n_estimators'] = list(range(1, 100, 10))
rf_space['class_weight'] = [None, 'balanced', 'balanced_subsample']
rf_space['max_features'] = [None, 'auto', 'sqrt', 'log2']

rf_search = RandomizedSearchCV(rf_model, rf_space, 
                                    n_iter=10, scoring='f1', cv=rf_cv, random_state=1)

In [20]:
from datetime import datetime

In [21]:
start = datetime.now()
rf_result = rf_search.fit(features, labels)
end = datetime.now()
print(end - start)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

2:48:44.272427


In [23]:
end, start

(datetime.datetime(2023, 10, 9, 13, 19, 3, 260869),
 datetime.datetime(2023, 10, 9, 10, 30, 18, 988442))

In [24]:
# summarize result
print('Best Score: %s' % rf_result.best_score_)
print('Best Hyperparameters: %s' % rf_result.best_params_)

Best Score: 0.9085199276170173
Best Hyperparameters: {'random_state': 1, 'n_estimators': 81, 'max_features': 'log2', 'max_depth': 11, 'criterion': 'entropy', 'class_weight': 'balanced'}


In [9]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [12]:
rf_clf = RandomForestClassifier(criterion='entropy', random_state=1,
                                n_estimators=81, max_features='log2', max_depth=11, class_weight='balanced')

In [13]:
rf_clf.fit(train_features, train_labels)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=11, max_features='log2', n_estimators=81,
                       random_state=1)

In [14]:
predictions = rf_clf.predict(test_features)
probs = rf_clf.predict_proba(test_features)[:, 1]

In [15]:
result_table = [["F1", "Accuracy", "AUC"]]

In [16]:
result_table.append([
    round(f1_score(test_labels, predictions), 3),
    round(accuracy_score(test_labels, predictions), 3),
    round(roc_auc_score(test_labels, probs), 3),
])

In [31]:
result_table

[['F1', 'Accuracy', 'AUC'], [0.91, 0.94, 0.984]]

In [32]:
from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

+------+------------+-------+
|   F1 |   Accuracy |   AUC |
+======+============+=======+
| 0.91 |       0.94 | 0.984 |
+------+------------+-------+


In [ ]:
b

In [17]:

from tabulate import tabulate
print(tabulate(result_table, headers="firstrow", tablefmt="grid"))

+-------+------------+-------+
|    F1 |   Accuracy |   AUC |
+=======+============+=======+
| 0.902 |      0.934 | 0.983 |
+-------+------------+-------+
